In [59]:
import random

from deap import base
from deap import creator
from deap import tools

Objective: We want to generate a bit pattern of length NUM_BITS, and we want it to contain TARGET ones. 

In [78]:
NUM_BITS = 100
TARGET = 100
MAX_GENERATIONS = 1000
NUM_INDIVIDUALS = 300

# We need to define an evaluation function that can be used to target this objective.
#def eval_func(individual):
 #   return len(individual) - abs(sum(individual) - TARGET)

# The goal ('fitness') function to be maximized
# number of 1's in th bitstream
def evalOneMax(individual):
    return sum(individual),

In [80]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

In [81]:
# Attribute generator 
#                      define 'attr_bool' to be an attribute ('gene')
#                      which corresponds to integers sampled uniformly
#                      from the range [0,1] (i.e. 0 or 1 with equal
#                      probability)
toolbox.register("attr_bool", random.randint, 0, 1)

In [82]:
# Structure initializers
#                         define 'individual' to be an individual
#                         consisting of 100 'attr_bool' elements ('genes')
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_bool, NUM_BITS)

# define the population to be a list of individuals
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [83]:
# Operator registration
#----------
# register the goal / fitness function
toolbox.register("evaluate", evalOneMax)

# register the crossover operator
toolbox.register("mate", tools.cxTwoPoint)

# register a mutation operator with a probability to
# flip each attribute/gene of 0.05
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)

# operator for selecting individuals for breeding the next
# generation: each individual of the current generation
# is replaced by the 'fittest' (best) of three individuals
# drawn randomly from the current generation.
toolbox.register("select", tools.selTournament, tournsize=3)

In [90]:
def main():
    random.seed(64)

    # create an initial population of individuals (where each individual is a list of integers)
    pop = toolbox.population(n=NUM_INDIVIDUALS)

    # CXPB  is the probability with which two individuals
    #       are crossed
    #
    # MUTPB is the probability for mutating an individual
    CXPB, MUTPB = 0.5, 0.2

    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    print("Start of evolution:  %i individuals" % len(pop))

    # Extracting all the fitnesses of 
    fits = [ind.fitness.values[0] for ind in pop]

    # Variable keeping track of the number of generations
    g = 0

    # Begin the evolution
    while max(fits) < TARGET and g < MAX_GENERATIONS:
        # A new generation
        g = g + 1
        print("\tGeneration %i --" % g)

        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):

            # cross two individuals with probability CXPB
            if random.random() < CXPB:
                toolbox.mate(child1, child2)

                # fitness values of the children
                # must be recalculated later
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:

            # mutate an individual with probability MUTPB
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        print("\tEvaluated %i invalid individuals" % len(invalid_ind))

        # The population is entirely replaced by the offspring
        pop[:] = offspring

        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]

        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5

        print("\tMin ", min(fits), " - Max ", max(fits))
        print("\tAvg ", round(mean,2), " - Std ", round(std,2))

    print("-- End of evolution --")

    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))


In [91]:
if __name__ == "__main__":
    main()


Start of evolution:  300 individuals
	Generation 1 --
	Evaluated 181 invalid individuals
	Min  44.0  - Max  66.0
	Avg  54.83  - Std  4.35
	Generation 2 --
	Evaluated 191 invalid individuals
	Min  47.0  - Max  68.0
	Avg  58.46  - Std  3.46
	Generation 3 --
	Evaluated 199 invalid individuals
	Min  52.0  - Max  68.0
	Avg  60.95  - Std  2.9
	Generation 4 --
	Evaluated 167 invalid individuals
	Min  47.0  - Max  71.0
	Avg  62.96  - Std  2.91
	Generation 5 --
	Evaluated 175 invalid individuals
	Min  57.0  - Max  73.0
	Avg  64.99  - Std  2.85
	Generation 6 --
	Evaluated 168 invalid individuals
	Min  58.0  - Max  74.0
	Avg  66.93  - Std  2.81
	Generation 7 --
	Evaluated 187 invalid individuals
	Min  59.0  - Max  76.0
	Avg  68.92  - Std  2.83
	Generation 8 --
	Evaluated 171 invalid individuals
	Min  62.0  - Max  76.0
	Avg  70.89  - Std  2.45
	Generation 9 --
	Evaluated 155 invalid individuals
	Min  62.0  - Max  80.0
	Avg  72.69  - Std  2.62
	Generation 10 --
	Evaluated 171 invalid individuals
	M